# Demo for LumiSpy package working with Gatan SEM CL data

This notebook shows:

- Loading a `.dm4/.dm3` file correctly with the LumiSpy load function
- Plotting cathodoluminescence data in different ways
- Correcting for spectral artefacts

Import packages:

In [ ]:
%matplotlib widget
# Use '%matplotlib widget' in JupterLab and '%matplotlib notebook' in JupyterNotebook for interactive inline functionality (e.g. on binder)
#For pop-up window plots on your local computer, use '%matplotlib tk' or '%matplotlib qt' instead
import lumispy as lum
import hyperspy.api as hs

## Loading .dm4/dm3 files

Use the HyperSpy `load` function to load Gatan cathodoluminescence spectrum or spectrum image files.
State the relative path to the `Test-spectrum_ZnO-NWs.dm4` file.

In [ ]:
# Single spectrum
cl_spec = hs.load('demo-files/load_from_GatanFiles/Test-spectrum_ZnO-NWs.dm4')

The `signal_type` is automatically recognized as `CL` and the data is saved as a `CLSpectrum` object.

In [ ]:
cl_spec

If you prefer to use the `CLSEMSpectrum` subclass, you can change the signal_type:

In [ ]:
cl_spec.set_signal_type('CL_SEM')

In [ ]:
cl_spec

In [ ]:
cl_spec.plot()

#### Convert from wavelength to energy axis

Required versions: hyperspy>=1.7.0 and lumispy>=0.2

*LumiSpy* provides easy conversions of the signal axis to the **energy scale**:

It can either replace the axis in the existing object (default) or create a copy of the signal object with the new axis (`inplace=False`):

In [ ]:
cl_spec_eV = cl_spec.to_eV(inplace=False)

In [ ]:
cl_spec_eV.plot()

### Loading a spectral image

Loads a linescan (navigation dimension = 1 and signal dimension = 1)

In [ ]:
cl_linescan = hs.load('demo-files/load_from_GatanFiles/Test-linescan_InGaAs-NWs.dm4')

Plot the data, to change the position of the displayed spectrum, drag the line on the navigator or use `left` and `right` arrow keys:

In [ ]:
cl_linescan.plot()

## Correction of spectral defects

### Remove background (interactive)

HyperSpy has an interactive tool for **background removal** that supports various functions, let's start by removing a **simple offset**:
1. Select a region to be used to determine the background: On the signal plot click, drag and release
2. You can still move the region or it's boundaries with the mouse and inspect the different spectra using the navigator to make sure the region is right
3. Press `Apply`

In [ ]:
cl_linescan.remove_background(background_type="Offset")

### Remove last pixels from the spectrum

The signal drops to negative values during the last pixels, so lets remove the last three pixels from every spectrum:

In [ ]:
cl_linescan = cl_linescan.isig[:-3]
cl_linescan.plot()

### Remove spikes (interactive)

There is also a tool for interactive removal of cosmic rays (pixels with sharp spikes), see `Help` for instructions.

In brief:
- Inspect the derivate histogram
- Set a sensible threshold to catch the outliers in the histogram
- Iterate through `Find next` / `Remove spike` to continue for wrong identifications / remove identified spikes
- `Close` when finished

NOTE: This interactive function does not work with inline plotting (e.g. on binder)

TODO: choose data file which includes several spikes

In [ ]:
cl_linescan.spikes_removal_tool()

## Working with the metadata

All the microscope & some spectrometer parameters are saved in the metadata (note that currently the Gatan microscope parameters are saved under `TEM` even if acquired with an SEM).

Metadata can be accessed via (you can expand/close the nodes by clicking on them):

In [ ]:
cl_spec.metadata

The original metadata from the `.dm4` file is available at `original_metadata`:

In [ ]:
cl_spec.original_metadata

You can add or change parameters manually using:

In [ ]:
cl_spec.metadata.set_item('General.title', 'demo spectrum')
cl_spec.metadata.set_item('General.comment', 'I love using LumiSpy')

In [ ]:
cl_spec.metadata

## Saving the manipulated data to the HyperSpy format

HyperSpy can not save to `.dm4` files, but supports writing files to the `hdf5` format:

In [ ]:
cl_spec.save('cl_test_spectrum.hspy')

## END